# Durr & Hoyer's Algorithm
### Minima/Maxima Finding Algorithm

### Introduction
Suppose you have an array of size *N* and an oracle such that, given *i*, it will gives you a superposition of all indices *j* such that *arr[j] < arr[i]*. Durr & Hoyer's algorithm can achieve this with $O(\sqrt{N}log(N))$.

In this notebook, we are not concerned about the implementation of such an oracle. The oracle would be built using *ControlledOnIntOracle()* from *Oracles.ipynb* notebook. The Durr & Hoyer's Algorithm is implemented at the end of the notebook.

### Importing Libraries

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from math import pi
import math
import random
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

from qiskit import IBMQ, Aer, BasicAer, execute
from qiskit.providers.ibmq import least_busy
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit.quantum_info import state_fidelity
from qiskit.tools.visualization import circuit_drawer
from qiskit.circuit.library.generalized_gates.mcmt import MCMT
from qiskit.circuit.library.standard_gates.x import XGate, HGate
from qiskit.circuit.library import ZGate
from qiskit.visualization import plot_histogram

### Defining Variables
The following array will be used throughout the notebook.
The next cell contains the function *f* which lowers the time complexity of the oracle.

In [15]:
arr = [22, 42, 2, 7, 74, 11, 86, 10, 3, 6, 93, 90, 23, 89, 92, 203, 84, 33, 32, 31, 40, 49, 84, 92]
n = len(arr)
print("The number of elements in the array is " + str(n) + ".")
simulator = Aer.get_backend('qasm_simulator')

The number of elements in the array is 24.


In [19]:
pairedarr = []
sortedind = []
for i in range(len(arr)):
    pairedarr.append((arr[i], i))
pairedarr.sort()
for i in range(len(pairedarr)):
    sortedind.append(pairedarr[i][1])

def f(x):
    for i in range(len(sortedind)):
        if (sortedind[i] == x):
            return sortedind[:i]
        
def converttoint(s):
    ans = 0
    for i in range(len(s)):
        ans = ans + (2**i)*int(s[len(s) - i - 1])
    return ans

### Template Oracles
The following oracles (*AndOracle()*, *ControlledOnIntOracle()*, *GroverOracle()*) from *Oracles.ipynb* are useful in the implementation of the *MinimumOracle()*.

In [17]:
def AndOracle(given_circuit, nn, target_qubit):
    given_circuit.barrier()
    controlled_xgate = XGate().control(nn)
    curlist = list(range(0, nn))
    curlist.append(target_qubit)
    given_circuit.append(controlled_xgate, curlist)
    given_circuit.barrier()


def ControlledOnIntOracle(given_circuit, nn, target_qubit, x, reverse=False):
    
    if (x != 0 and nn < int(math.floor(math.log2(x) + 1))):
        print("Not enough qubits to do ControlledOnIntOracle\n")
        return
    
    given_circuit.barrier()
    xx = x
    counter = 0
    for i in range(nn):
        if (xx%2 == 0):
            given_circuit.x(counter)
        xx = (xx - (xx%2))/2
        counter = counter + 1
    AndOracle(given_circuit, nn, target_qubit)
    
    xx = x
    counter = 0
    for i in range(nn):
        if (xx%2 == 0):
            given_circuit.x(counter)
        xx = (xx - (xx%2))/2
        counter = counter + 1
    given_circuit.barrier()

    
def DiffusionOperator(given_circuit, nn, name = "DiffusionOperator"):
    given_circuit.h(range(nn))
    given_circuit.x(range(nn))
    controlled_gate = ZGate().control(nn - 1)
    given_circuit.append(controlled_gate, list(range(0, nn)))
    given_circuit.x(range(nn))
    given_circuit.h(range(nn))

    
def GroverOracle(given_circuit, nn, indices_to_mark, name = 'GroverOracle'):
    for i in indices_to_mark:
        ControlledOnIntOracle(given_circuit, nn, nn, i)


## Minimum Oracle

The *MinimumOracle()* is implemented such that all the 'good' indices are marked by *GroverOracle()*. In the oracle, the amplitude amplification process is also done. This oracle now returns a superposition of all indices *j* such that *arr[j] < arr[i]*.

Although we are not investigating the time complexity of the oracle, it is worth noting that classically, the cost of querying the oracle is $O(N)$, because all indices which satisfies *arr[j] < arr[i]* must be returned to the oracle.

In [25]:
def MinimumOracle(given_circuit, nn, target_qubit, x, reverse=False, name = "MinimumOracle"):
    print("Called Minimum Oracle with index " + str(x) + ". arr[x] = " + str(arr[x]) + ".")
    
    cur_array = f(x)
    num = len(f(x))
    print("Oracle says in a deep and mysterious voice: " + str(cur_array) + ".")
    
    if (num == 0):
        return False
    r = int(np.floor(np.pi/4*np.sqrt(2**nn/num)))
    for _ in range(r):
        GroverOracle(given_circuit, nn, cur_array)
        DiffusionOperator(given_circuit, nn)
    return True

In [36]:
# nn = number of qubits, excluding the ancilla qubit
# x = number of items in the array
def DurrHoyerAlgorithm(nn, x):
    
    callstooracle = 0
    
    # Step 1: Generates a random index
    y = random.randint(0, x - 1)
    done = True
    
    print("Initial Guess: " + str(y) + ".")
    
    while (done == True):
        durrhoyer_q = QuantumRegister(nn + 1)
        durrhoyer_c = ClassicalRegister(nn)
        durrhoyer_qc = QuantumCircuit(durrhoyer_q, durrhoyer_c, name = "DurrHoyer Circuit")

        # Step 2: Apply Hadamard gate to a superposition of all possible states
        durrhoyer_qc.h(range(nn))

        # Step 3: Call the Minimum Oracle
        done = MinimumOracle(durrhoyer_qc, nn, nn, y)
        callstooracle = callstooracle + 1
        
        if (done == False):
            counts = execute(durrhoyer_qc, simulator, shots = 200).result().get_counts(durrhoyer_qc)
            return (y, callstooracle, counts)
        
        # Step 4: Measure and get a value with the most occurrence
        durrhoyer_qc.measure(range(nn), range(nn))
        counts = execute(durrhoyer_qc, simulator, shots = 200).result().get_counts(durrhoyer_qc)
        sortedcounts = {k: v for k, v in sorted(counts.items(), key=lambda item: item[1])}

        curlist = list(sortedcounts.keys())
        yprime = 0
        for i in range(len(curlist)):
            if (converttoint(curlist[-1 - i]) >= x):
                continue
            yprime = converttoint(curlist[-1-i])
            break
    
        print("Next Guess: " + str(yprime) + ".\n")

        if (arr[yprime] < arr[y]):
            y = yprime
        del durrhoyer_qc
    
    return (y, callstooracle, counts)

In [44]:
newn = 1 + math.floor(math.log2(n))
(min_index, num_calls, final_count) = DurrHoyerAlgorithm(newn, n)
print("The minimum index is " + str(min_index) + " with value arr[x] of " + str(arr[min_index]) + ".")
print("The number of calls made to the minimum oracle is " + str(num_calls) + ".")

Initial Guess: 20.
Called Minimum Oracle with index 20. arr[x] = 40.
Oracle says in a deep and mysterious voice: [2, 8, 9, 3, 7, 5, 0, 12, 19, 18, 17].
Next Guess: 2.

Called Minimum Oracle with index 2. arr[x] = 2.
Oracle says in a deep and mysterious voice: [].
The minimum index is 2 with value arr[x] of 2.
The number of calls made to the minimum oracle is 2.


### Personal Reflection
As I implemented the whole notebook, I constantly had some doubts about the quantum algorithms, especially how they compared to classical algorithms. Below are some of my thoughts.

1. I was initially confused at why the time complexity is $O(\sqrt{N})$. I thought the time complexity was $O(log(N))$ instead, because it takes an expected $log(N)$ steps to reach the minimum index. My mistake was that I didn't take into account the time complexity for amplitude amplification, which was $O(\sqrt{N})$. Hence, combined time complexity is $O(\sqrt{N}log(N))$.

2. I wondered to myself: Suppose I have a classical oracle that gives me all the indices. Then, by randomly picking one index and querying the oracle again, I would take an expected $O(log(N))$ steps to reach the minimum. So how does this quantum algorithm offer me superiority in complexity with the $\sqrt{N}$ factor? I still haven't quite figured this out, but my thinking is that classically, such oracle would take at least $O(N)$ time complexity to query, while for quantum there exists a faster way if the database had already been converted to quantum.

3. Currently, the complexity of creating the *MinimumOracle()* is $O(N)$ and cost of querying is $O(N)$. Is there a better complexity in the quantum realm?

### References

1. http://cds.cern.ch/record/408677/files/9911082.pdf